# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.60/12512/1 Dashboard: http://10.20.0.60:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,999,Laura,0.7356810631685795,-0.253595711396603
2000-01-01 00:00:01,999,Tim,0.7825769742982347,-0.18206032379922488
2000-01-01 00:00:02,1025,Ray,0.76307680491423,-0.7338850286317098
2000-01-01 00:00:03,996,Hannah,0.5331947698086987,0.27157646195175755
2000-01-01 00:00:04,1017,Yvonne,-0.11884975816069132,-0.2637328294550776
2000-01-01 00:00:05,987,Edith,0.15857705395494803,0.7644488725970358
2000-01-01 00:00:06,956,Michael,0.9949084673583661,0.8512388723061775
2000-01-01 00:00:07,1007,Tim,-0.9352739595540314,-0.9613860730331081
2000-01-01 00:00:08,993,Jerry,-0.9360014842468083,0.934288014327292


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1040,Laura,0.9671042575419229,-0.18726327931122322
2000-01-30 00:00:01,973,Xavier,-0.8712225118612713,0.17533744933879625
2000-01-30 00:00:02,973,Sarah,-0.3496208706667898,0.4070906738267781
2000-01-30 00:00:03,996,Tim,-0.5331308240933894,0.6162646929880742
2000-01-30 00:00:04,1020,Frank,0.0566927743975556,0.2493432030486249
2000-01-30 00:00:05,1025,Kevin,-0.5231844374708725,0.2887710717784491
2000-01-30 00:00:06,1032,Dan,-0.07361403023743351,-0.23334320672369913
2000-01-30 00:00:07,981,Victor,0.14575502420264308,0.15269294544421474
2000-01-30 00:00:08,986,Tim,0.5173676338546529,-0.012417860361652266


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,999,Laura,0.735681,-0.253596
1,2000-01-01 00:00:01,999,Tim,0.782577,-0.182060
2,2000-01-01 00:00:02,1025,Ray,0.763077,-0.733885
3,2000-01-01 00:00:03,996,Hannah,0.533195,0.271576
4,2000-01-01 00:00:04,1017,Yvonne,-0.118850,-0.263733


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,999,Laura,0.735681,-0.253596
1,2000-01-01 00:00:01,999,Tim,0.782577,-0.182060
2,2000-01-01 00:00:02,1025,Ray,0.763077,-0.733885
3,2000-01-01 00:00:03,996,Hannah,0.533195,0.271576
4,2000-01-01 00:00:04,1017,Yvonne,-0.118850,-0.263733


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.72 s, sys: 565 ms, total: 5.28 s
Wall time: 3.48 s


name
Alice      -0.000455
Bob         0.001627
Charlie    -0.000603
Dan        -0.000173
Edith      -0.002154
Frank      -0.003351
George     -0.001979
Hannah      0.002002
Ingrid      0.003140
Jerry      -0.002037
Kevin       0.000370
Laura       0.003076
Michael    -0.001000
Norbert    -0.001281
Oliver      0.001159
Patricia   -0.003820
Quinn      -0.002070
Ray        -0.001234
Sarah      -0.003289
Tim         0.000708
Ursula     -0.002163
Victor      0.002525
Wendy       0.002288
Xavier      0.002258
Yvonne     -0.001211
Zelda       0.002486
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.44 s, sys: 248 ms, total: 1.68 s
Wall time: 1.13 s


name
Alice      -0.000455
Bob         0.001627
Charlie    -0.000603
Dan        -0.000173
Edith      -0.002154
Frank      -0.003351
George     -0.001979
Hannah      0.002002
Ingrid      0.003140
Jerry      -0.002037
Kevin       0.000370
Laura       0.003076
Michael    -0.001000
Norbert    -0.001281
Oliver      0.001159
Patricia   -0.003820
Quinn      -0.002070
Ray        -0.001234
Sarah      -0.003289
Tim         0.000708
Ursula     -0.002163
Victor      0.002525
Wendy       0.002288
Xavier      0.002258
Yvonne     -0.001211
Zelda       0.002486
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.34 s, sys: 95.6 ms, total: 1.43 s
Wall time: 990 ms


name
Alice      -0.000455
Bob         0.001627
Charlie    -0.000603
Dan        -0.000173
Edith      -0.002154
Frank      -0.003351
George     -0.001979
Hannah      0.002002
Ingrid      0.003140
Jerry      -0.002037
Kevin       0.000370
Laura       0.003076
Michael    -0.001000
Norbert    -0.001281
Oliver      0.001159
Patricia   -0.003820
Quinn      -0.002070
Ray        -0.001234
Sarah      -0.003289
Tim         0.000708
Ursula     -0.002163
Victor      0.002525
Wendy       0.002288
Xavier      0.002258
Yvonne     -0.001211
Zelda       0.002486
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html